In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LSTM
from keras.optimizers import Adam
import xgboost as xgb
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the data
inventory_data = pd.read_csv('Domestic Auto Inventories.csv', index_col='DATE', parse_dates=True)
production_data = pd.read_csv('Domestic Auto Production.csv', index_col='DATE', parse_dates=True)
sales_data = pd.read_csv('Total Vehicle Sales.csv', index_col='DATE', parse_dates=True)

# Combine the data into a single DataFrame
df = pd.concat([inventory_data, production_data, sales_data], axis=1)

# Fill missing values with forward fill first, then backward fill
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)

In [ ]:
# Define functions to train models and calculate metrics
def train_models_for_sku(sku_data):
    train, test = sku_data[sku_data.index < '2024-01-01'], sku_data[sku_data.index >= '2024-01-01']
    
    # Train each expert model on the training data
    arima_model = ARIMA(train, order=(5, 1, 0)).fit()
    ets_model = ExponentialSmoothing(train, trend='add', seasonal=None).fit()
    X = np.arange(len(train)).reshape(-1, 1)
    y = train.values
    rf_model = RandomForestRegressor(n_estimators=100).fit(X, y)
    
    # Generate predictions on training data
    arima_forecast_train = arima_model.predict(start=train.index[0], end=train.index[-1])
    ets_forecast_train = ets_model.predict(start=train.index[0], end=train.index[-1])
    rf_forecast_train = rf_model.predict(X)
    
    # Generate predictions on test data
    arima_forecast_test = arima_model.forecast(steps=len(test))
    ets_forecast_test = ets_model.forecast(steps=len(test))
    rf_forecast_test = rf_model.predict(np.arange(len(train), len(train) + len(test)).reshape(-1, 1))
    
    return arima_forecast_train, ets_forecast_train, rf_forecast_train, arima_forecast_test, ets_forecast_test, rf_forecast_test, train, test

def calculate_metrics(true_values, predicted_values):
    mae = mean_absolute_error(true_values, predicted_values)
    mape = mean_absolute_percentage_error(true_values, predicted_values)
    wape = np.sum(np.abs(true_values - predicted_values)) / np.sum(np.abs(true_values))
    return mae, mape, wape

# Example: Prepare data for a single SKU (column)
sku_data = df.iloc[:, 0]  # Assuming SKU is the first column

# Train models and get forecasts
arima_forecast_train, ets_forecast_train, rf_forecast_train, arima_forecast_test, ets_forecast_test, rf_forecast_test, train, test = train_models_for_sku(sku_data)

In [ ]:
# Simplified gating network definition
def build_gating_network(input_dim, num_experts):
    input_layer = Input(shape=(input_dim,))
    output_layer = Dense(num_experts, activation='softmax')(input_layer)
    gating_model = Model(inputs=input_layer, outputs=output_layer)
    return gating_model

# Combine forecasts on training data
combined_forecasts_train = np.vstack((arima_forecast_train, ets_forecast_train, rf_forecast_train)).T

# Train the gating network using expert predictions on training data
gating_network = build_gating_network(combined_forecasts_train.shape[1], num_experts=3)
gating_network.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
gating_network.fit(combined_forecasts_train, train, epochs=50, batch_size=32, verbose=1)

In [ ]:
# Combine forecasts on test data
combined_forecasts_test = np.vstack((arima_forecast_test, ets_forecast_test, rf_forecast_test)).T

# Make predictions on the test data using the trained gating network
gating_weights_test = gating_network.predict(combined_forecasts_test)

# Calculate the final MoE predictions as a weighted sum of expert predictions on test data
moe_predictions_test = np.sum(gating_weights_test * combined_forecasts_test, axis=1)

# Evaluate the MoE model on the test set
mse_test = np.mean((moe_predictions_test - test) ** 2)
print(f'MoE Model MSE on test data: {mse_test:.4f}')